# Putting it all together

So far we have done the following on the prior Notebooks:

- **Notebook 01**: We loaded the Azure Search Engine with enriched PDFs in index: "cogsrch-index-files"
- **Notebook 02**: We added AzureOpenAI GPT models to enhance the the production of the answer by using Utility Chains of LLMs
- **Notebook 03**: We manually loaded an index with large/complex PDFs information , "cogsrch-index-books"
- **Notebook 04**: We added memory to our system in order to power a conversational Chat Bot
- **Notebook 05**: We introduced Agents and Tools and built the first Skill/Agent, that can do RAG over a search engine

We are missing one more thing: **How do we glue all these features together into a very smart GPT Smart Search Engine Chat Bot?**

We want a virtual assistant for our company that can get the question, think what tool to use, then get the answer. The goal is that, regardless of the source of the information (Search Engine, Bing Search, SQL Database, CSV File, JSON File, APIs, etc), the Assistant can answer the question correctly using the right tool.<br>
※ We can build more tools the agent will use.

In this Notebook we are going to create that "brain" Agent (also called Master Agent), that:

1) understands the question, interacts with the user 
2) talks to other specialized Agents that are connected to diferent sources
3) once it get's the answer it delivers it to the user or let the specialized Agent to deliver it directly

This is the same concept of [AutoGen](https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/): Agents talking to each other.

![image](./images/AutoGen_Fig1.png)

In [ ]:
import os
import random
import json
import requests
from operator import itemgetter
from typing import Union, List
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec, ConfigurableField
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

#custom libraries that we will use later in the app
from common.utils import (
    DocSearchAgent, 
    CSVTabularAgent, 
    SQLSearchAgent, 
    ChatGPTTool, 
    BingSearchAgent, 
    APISearchAgent, 
    reduce_openapi_spec
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))


In [ ]:
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

### Get the Tool - DocSearch Agent, ChatGPT (more agents can be included - CSV Agent, SQL Agent, Web Search Agent, ChatGPT, API Agent)

**Consider the following concept:** Agents, which are essentially software entities designed to perform specific tasks, can be equipped with tools. These tools themselves can be other agents, each possessing their own set of tools. This creates a layered structure where tools can range from code sequences to human actions, forming interconnected chains. Ultimately, you're constructing a network of agents and their respective tools, all collaboratively working towards solving a specific task (This is what ChatGPT is). This network operates by leveraging the unique capabilities of each agent and tool, creating a dynamic and efficient system for task resolution.

In the file `common/utils.py` we created Agent Tools Classes for each of the Functionalities that we developed in prior Notebooks. 

In [ ]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

# We can run the everything with GPT3.5, but try also GPT4 and see the difference in the quality of responses
# You will notice that GPT3.5 is not as reliable.

llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], 
                      temperature=0, max_tokens=COMPLETION_TOKENS)

# Uncomment below if you want to see the answers streaming
# llm = AzureChatOpenAI(deployment_name=os.environ["GPT35_DEPLOYMENT_NAME"], temperature=0.5, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager)


In [ ]:
doc_indexes = ["cogsrch-index-files"]
doc_search = DocSearchAgent(llm=llm, indexes=doc_indexes,
                           k=6, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="docsearch",
                           description="useful when the questions includes the term: docsearch",
                           callback_manager=cb_manager, verbose=False)

In [ ]:
book_indexes = ["cogsrch-index-books"]
book_search = DocSearchAgent(llm=llm, indexes=book_indexes,
                           k=5, reranker_th=1,
                           sas_token=os.environ['BLOB_SAS_TOKEN'],
                           name="booksearch",
                           description="useful when the questions includes the term: booksearch",
                           callback_manager=cb_manager, verbose=False)

In [ ]:
## ChatGPTTool is a custom Tool class created to talk to ChatGPT knowledge
chatgpt_search = ChatGPTTool(llm=llm, callback_manager=cb_manager,
                             name="chatgpt",
                            description="use for general questions, profile, greeting-like questions and when the questions includes the term: chatgpt",
                            verbose=False)

### Variables/knobs to use for customization

As you have seen so far, there are many knobs that you can dial up or down in order to change the behavior of your GPT Smart Search engine application, these are the variables you can tune:

- <u>llm</u>:
  - **deployment_name**: this is the deployment name of your Azure OpenAI model. This of course dictates the level of reasoning and the amount of tokens available for the conversation. For a production system you will need gpt-4-32k. This is the model that will give you enough reasoning power to work with agents, and enough tokens to work with detailed answers and conversation memory.
  - **temperature**: How creative you want your responses to be
  - **max_tokens**: How long you want your responses to be. It is recommended a minimum of 500
- <u>Tools</u>: To each tool you can add the following parameters to modify the defaults (set in utils.py), these are very important since they are part of the system prompt and determines what tool to use and when.
  - **name**: the name of the tool
  - **description**: when the brain agent should use this tool
- <u>DocSearchAgent</u>: 
  - **k**: The top k results per index from the text search action
  - **similarity_k**: top k results combined from the vector search action
  - **reranker_th**: threshold of the semantic search reranker. Picks results that are above the threshold. Max possible score=4
  
in `utils.py` you can also tune:
- <u>model_tokens_limit</u>: In this function you can edit what is the maximum allows of tokens reserve for the content. Remember that the remaining will be for the system prompt plus the answer

### Test the Tools

In [ ]:
# Test the Document Search Tool with a question that we know it has the answer for
printmd(doc_search.run("what is the responsibility of Manager of Human Resources?"))

In [ ]:
# Test the other index created manually
printmd(book_search.run("Can I move, backup, and restore indexes?"))

In [ ]:
# Test the ChatGPTWrapper Search Tool
printmd(chatgpt_search.run("what is the function in python that allows me to get a random number?"))

### Define what tools are we going to give to our brain agent

Go to `common/utils.py` to check the tools definition and the instructions on what tool to use when

In [ ]:
tools = [doc_search, book_search, chatgpt_search]

# Option 1: Using OpenAI functions as router

We need a method to route the question to the right tool, one way to do this is to use OpenAI models functions via the Tools API (models 1106 and newer). To do this, we need to bind these tools/functions to the model and let the model respond with the right tool to use.

The advantage of this option is that there is no another agent in the middle between the experts (agent tools) and the user. Each agent tool responds directly. Also, another advantage is that multiple tools can be called in parallel.

**Note**: on this method it is important that each agent tool has the same system profile prompt so they adhere to the same reponse guidelines.

In [ ]:
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}

In [ ]:
def call_tool(tool_invocation: dict) -> Union[str, Runnable]:
    """Function for dynamically constructing the end of the chain based on the model-selected tool."""
    tool = tool_map[tool_invocation["type"]]
    return RunnablePassthrough.assign(output=itemgetter("args") | tool)

def print_response(result: List):
    for answer in result:
        printmd("**"+answer["type"] + "**" + ": " + answer["output"])
        printmd("----")
      
# .map() allows us to apply a function to a list of inputs.
call_tool_list = RunnableLambda(call_tool).map()
agent = llm_with_tools | JsonOutputToolsParser() | call_tool_list

In [ ]:
result = agent.invoke("hi, how are you, what is your name?")
print_response(result)

In [ ]:
result = agent.invoke("Who is the current president of France?")
print_response(result)

In [ ]:
result = agent.invoke("docsearch,chatgpt, what is the responsibility of Manager of Human Resources?")
print_response(result)

# Option 2: Using a user facing agent that calls the agent tools experts

With this method, we create a user facing agent that talks to the user and also talks to the experts (agent tools)

### Initialize the brain agent

In [ ]:
agent = create_openai_tools_agent(llm, tools, CUSTOM_CHATBOT_PROMPT)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [ ]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos


In [ ]:
brain_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [ ]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
print(random_session_id, ramdom_user_id)

### Let's talk to our GPT Smart Search Engine chat bot now

In [ ]:
# This question should not use any tool, the brain agent should answer it without the use of any tool
printmd(brain_agent_executor.invoke({"question": "Hi, I'm Pablo Marin, how are you doing today?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "what is your name and what do you do?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "booksearch, Can I move indexes?"}, 
                                    config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "chatgpt, tell me the formula in physics for momentum"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "docsearch, what is the responsibility of Manager of Human Resources?"}, config=config)["output"])

In [ ]:
printmd(brain_agent_executor.invoke({"question": "Thank you Jarvis!"}, config=config)["output"])

### Let's talk to our GPT Smart Search Engine chat bot with more questions and validate the responses from the chat bot

# Summary

Great!, We just built the GPT Smart Search Engine!
In this Notebook we created the brain, the decision making Agent that decides what Tool to use to answer the question from the user. This is what was necessary in order to have an smart chat bot.

We can have many tools to accomplish different tasks, including connecting to APIs, dealing with File Systems, and even using Humans as Tools. For more reference see [HERE](https://python.langchain.com/docs/integrations/tools/)

# NEXT
It is time now to use all the functions and prompts build so far and build a Web application.
The Next notebook will guide you on how to build:

1) A Bot API Backend
2) A Frontend UI with a Search and Webchat interfaces